In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import torchvision
import Enneaception
from helper_functions import *

In [2]:
import sys, os, glob
import configparser
import time
import numpy as np
import torch
from time_left import pretty_time_left, pretty_print_time
from RetinaCheckerPandas import RetinaCheckerPandas
from helper_functions import initialize_meters, save_performance
from make_default_config import get_config
import inception_short
import torch.nn as nn

In [4]:
config = configparser.ConfigParser()
config.read('test2.cfg')
config['files']['image size'] = "897"
config['hyperparameter']['batch size'] = '1'

# create the checker class and initialize internal variables
rc = RetinaCheckerPandas()
rc.initialize(config)

rc.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Loading data sets based on configuration and enable normaization
rc.load_datasets()

# Initializing sampler and data (=patch) loader
rc.create_dataloader(0)

# Initialize the model
ckpt = torch.load('D:/ckpt/model_181209_seoul_rand_xs_best_accuracy.ckpt', map_location=rc.device)
rc.model = Enneaception.enneaception(pretrained_inception_dict=ckpt['state_dict'], num_classes=7)
for param in rc.model.inception_xs.parameters():
    param.requires_grad = False
rc.model = rc.model.to(rc.device)
rc.initialize_criterion()
rc.initialize_optimizer()

print('Model set up. Ready to train.')
print(rc)

Using device cpu
Model set up. Ready to train.
RetinaChecker
Network: inception_v3
Optimizer: Adam
Criterion: BCEWithLogitsLoss
Epoch: 0
Training root: D:/Dropbox/Data/retina_data/images/KU-test-cropped
Training file: D:/Dropbox/Data/retina_data/images/KUtest.csv
Dataset PandasDataset
    Number of datapoints: 135
    Root Location: D:/Dropbox/Data/retina_data/images/KU-test-cropped
    Transforms (if any): Compose(
                             RandomApply(
                             p=0.5
                             ColorJitter(brightness=0.0, contrast=0.0, saturation=0.0, hue=0.0)
                         )
                             RandomHorizontalFlip(p=0.5)
                             RandomVerticalFlip(p=0.5)
                             RandomChoice(
                             RandomResizedCrop(size=(897, 897), scale=(0.25, 1.0), ratio=(1, 1), interpolation=PIL.Image.BILINEAR)
                             RandomResizedCrop(size=(897, 897), scale=(0.25, 0.4), ratio=(0.8,

In [5]:
# Performance meters initalized (either empty or from file)
num_epochs = 20
train_loss, train_accuracy, test_loss, test_accuracy, test_confusion = initialize_meters( config, rc.start_epoch, num_epochs, 2 )

# Starting training & evaluation
start_time = time.time()
best_accuracy = 0
best_accuracy_2class = 0
best_sensitivity = 0
best_specificity = 0
for epoch in range(rc.start_epoch, num_epochs):
    start_time_epoch = time.time()

    # Train the model and record training loss & accuracy
    losses, accuracy = rc.train()

    # Logging and performance 
    train_loss[epoch] = losses.avg
    train_accuracy[epoch] = accuracy.avg

    # Validation
    losses, accuracy, confusion = rc.validate()

    # Logging and performance measuring overhead
    test_loss[epoch] = losses.avg
    test_accuracy[epoch] = accuracy.avg
    test_confusion[epoch,:,:] = confusion

    #print('Test Accuracy of the model on the {} test images: {} %'.format(accuracy.count, accuracy.avg*100))
    #print('Classes: {}'.format(rc.classes))
    print('Confusion matrix:\n', (confusion))

    #confusion_2class = reduce_to_2_classes( confusion, [(0,1), (2,3,4)])
    confusion_2class = confusion
    accuracy_2class = np.diag(confusion_2class).sum()/confusion_2class.sum()
    sensitivity_2class = confusion_2class[1,1]/confusion_2class[:,1].sum()
    specificity_2class = confusion_2class[0,0]/confusion_2class[:,0].sum()
    print('Accuracy: {:.1f}%'.format(accuracy_2class*100))
    print('Sensitivity: {:.1f}%'.format(sensitivity_2class*100))
    print('Specificity: {:.1f}%'.format(specificity_2class*100))

Epoch learning completed. Training accuracy 43.8%
Confusion matrix:
 tensor([[0., 4.],
        [6., 5.]])
Accuracy: 33.3%
Sensitivity: 55.6%
Specificity: 0.0%
Epoch learning completed. Training accuracy 43.8%
Confusion matrix:
 tensor([[1., 3.],
        [5., 6.]])
Accuracy: 46.7%
Sensitivity: 66.7%
Specificity: 16.7%
Epoch learning completed. Training accuracy 62.5%
Confusion matrix:
 tensor([[0., 2.],
        [6., 7.]])
Accuracy: 46.7%
Sensitivity: 77.8%
Specificity: 0.0%
Epoch learning completed. Training accuracy 43.8%
Confusion matrix:
 tensor([[0., 1.],
        [6., 8.]])
Accuracy: 53.3%
Sensitivity: 88.9%
Specificity: 0.0%
Epoch learning completed. Training accuracy 62.5%
Confusion matrix:
 tensor([[0., 1.],
        [6., 8.]])
Accuracy: 53.3%
Sensitivity: 88.9%
Specificity: 0.0%
Epoch learning completed. Training accuracy 43.8%
Confusion matrix:
 tensor([[2., 3.],
        [4., 6.]])
Accuracy: 53.3%
Sensitivity: 66.7%
Specificity: 33.3%
Epoch learning completed. Training accuracy 